In [ ]:
import findspark
findspark.init()

from pyspark import SparkConf,SparkContext
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext
from pyspark.sql import Row,SQLContext
from pyspark.sql.functions import arrays_zip, col,explode
import sys
import requests

In [ ]:
def getSparkSessionInstance(sparkConf):
    if ('sparkSessionSingletonInstance' not in globals()):
        globals()['sparkSessionSingletonInstance'] = SparkSession\
            .builder\
            .config(conf=sparkConf)\
            .getOrCreate()
    return globals()['sparkSessionSingletonInstance']

config = SparkConf().set('spark.driver.host','127.0.0.1').setAppName("NewsStreamingApp").setMaster("local[2]")
sc = SparkContext.getOrCreate(config)
# create the Streaming Context from the above spark context with window size 2 seconds
ssc = StreamingContext(sc, 2)
# read data from port 5000
dataStream = ssc.socketTextStream("127.0.0.1",5000)

In [ ]:
def readMyStream(rdd):
    spark = getSparkSessionInstance(rdd.context.getConf())
    if not rdd.isEmpty():
        liveData = spark.read.json(rdd)
        liveData.show()
        with open('livedata.csv', 'a+', encoding="utf-8-sig", newline='') as file:
            liveData.toPandas().to_csv(file, header=False,index=False)

dataStream.foreachRDD( lambda rdd: readMyStream(rdd) )

words = dataStream.flatMap(lambda line: line.split(" "))
words.pprint()

ssc.start()
# wait for the streaming to finish
ssc.awaitTermination()

In [ ]:
sc.stop()